In [ ]:
import numpy as np
from pathlib import Path
import os
import scipy
import pandas as pd

In [ ]:
folder_path = r"S:\microscopy\Calcium imaging\24.06.28_1361_5mM_aldi\analysis_replicate\papaer_analysis"
file_name = 'grouped_intensity_and_speed_df_merge.csv'
#strain_name = '2727'
#strain_file_name =  file_name + strain_name + ".csv"
worm_data_df = pd.read_csv(os.path.join(folder_path, file_name))
worm_data_df

In [ ]:
worm_data_df['centroid_x'] = pd.to_numeric(worm_data_df['centroid_x'], errors='coerce')
worm_data_df['centroid_y'] = pd.to_numeric(worm_data_df['centroid_y'], errors='coerce')

# Drop rows with NaN values in 'centroid_x' or 'centroid_y'
worm_data_df = worm_data_df.dropna(subset=['centroid_x', 'centroid_y'])

# Group by 'n_worm' and calculate distances within each group
def calculate_distance(group):
    x_squared_diff = (group['centroid_x'] - group['centroid_x'].shift()) ** 2
    y_squared_diff = (group['centroid_y'] - group['centroid_y'].shift()) ** 2
    group['distance'] = (x_squared_diff + y_squared_diff) ** 0.5
    return group

# Apply the function to each group
worm_data_df = worm_data_df.groupby((['n_worm', 'replicate']), group_keys=False).apply(calculate_distance)

# Define a speed coefficient µm/px * FPS
speed_coefficient = 0.73 * 0.17

# Calculate the speed by multiplying the distance by the speed coefficient
worm_data_df['speed'] = worm_data_df['distance'] * speed_coefficient
worm_data_df

In [ ]:
grouped_worms = worm_data_df.groupby(['replicate','n_worm'])
sorted_groups = grouped_worms.apply(lambda x: x.sort_values(by='time')).reset_index(drop=True)
regrouped_sorted = sorted_groups.groupby(['replicate', 'n_worm'])

# Interpolate
worm_data_df['speed'] = regrouped_sorted['speed'].transform(lambda x: x.interpolate(method='linear', limit_direction='both'))
grouped_worms = worm_data_df.groupby(['replicate','n_worm'])


worm_data_df['smooth_speed'] = grouped_worms['speed'].transform(lambda x: scipy.signal.savgol_filter(x, 20, 0, mode='mirror'))
worm_data_df

In [ ]:
grouped = worm_data_df.groupby(['time'])

# Now, calculate the mean of 'mean_intensity' and 'normalized_intensity' for each group
result = grouped.agg(
    w_count=('speed', 'count'),
    mean_speed=('speed', 'mean'),
    std_speed=('speed', 'std'),
    mean_speed_smooth=('smooth_speed', 'mean'),
    std_speed_smooth=('smooth_speed', 'std'),
    ).reset_index()

# Add standard error as a new column
result['speed_err'] = result['std_speed'] / np.sqrt(result['w_count'])
result['speed_err_smooth'] = result['std_speed_smooth'] / np.sqrt(result['w_count'])

result

In [ ]:
result.to_csv(Path(folder_path).joinpath(f'{file_name}_speed_mean.csv'), index=False) #False exclude index column

In [ ]:
# Filter and print speed values where time is 0
speeds_at_time_zero = worm_data_df[worm_data_df["time"] == 0]["speed"]
print(speeds_at_time_zero.tolist())  # Prints as a list

In [ ]:
worm_data_df["n_worm_replicate"] = worm_data_df["n_worm"].astype(str) + "_rep" + worm_data_df["replicate"].astype(str)

# Pivot the table
df_pivot = worm_data_df.pivot(index="time", columns="n_worm_replicate", values="speed")

# Rename columns for clarity
df_pivot.columns = [f"Worm_{col}_Speed" for col in df_pivot.columns]

# Display the new table
print(df_pivot.head())

In [ ]:
df_pivot.to_csv(Path(folder_path).joinpath(f'{file_name}_table_wormspeed.csv'), index=False) #False exclude index column

In [ ]:
group_worm = worm_data_df.groupby(['replicate', 'n_worm'])
# Now, calculate the mean of 'mean_intensity' and 'normalized_intensity' for each group
smooth_mean_worm = group_worm['smooth_speed'].mean()
smooth_mean_worm

In [ ]:
smooth_mean_worm.to_csv(Path(folder_path).joinpath(f'{file_name}_speed_for_stats.csv'), index=False)